# Scrittopoli - Giornata 1

In [1]:
%matplotlib inline

# Import the necessary packages and modules
import matplotlib
matplotlib.style.use('ggplot')
import matplotlib.pyplot as plt
import matplotlib.dates  as mpdt
import matplotlib.ticker as mptk
import matplotlib.mlab as mlab

import numpy as np
import pytz
mytz=pytz.timezone("Europe/Rome")
import pandas as pd
import datetime as dt
import math
import importlib
import random
import itertools
import csv
import string

import mylib.spreadsheet as ms
import mylib.elo as melo
import mylib.utility as mu


In [2]:
day="20171214_142354" # definitivo
calendario_csv="data/calendario_%s.csv" % day
gironi_csv="data/gironi_%s.csv" % day
tabellone_csv="data/tabellone_%s.csv" % day
ranking_csv="data/ranking_%s.csv" % day
scrittopoli_csv="data/scrittopoli_%s.csv" % day
elo_giocatori_csv="data/elo_giocatori_%s.csv" % day
elo_squadre_csv="data/elo_squadre_%s.csv" % day

calendario=pd.read_csv(calendario_csv,sep=":",quotechar='"',header=0,index_col=1).sort_index()
calendario=calendario.set_index(["girone","partita"], append=True)

gironi=pd.read_csv(gironi_csv,sep=":",quotechar='"',header=0)

tabellone=pd.read_csv(tabellone_csv,sep=",",quotechar='"',header=0,index_col=0)
elo_giocatori=pd.read_csv(elo_giocatori_csv,sep=",",quotechar='"',header=0,index_col=0)

ranking=pd.read_csv(ranking_csv,sep=",",quotechar='"',header=0)
ranking=ranking[ranking["squadra"]!="(riposo)"]
ranking=ranking.set_index("squadra")

scrittopoli=pd.read_csv(scrittopoli_csv,sep=",",quotechar='"',header=0)
scrittopoli=scrittopoli.set_index(["squadra","giocatore"])

elo_squadre=pd.read_csv(elo_squadre_csv,sep=",",quotechar='"',header=0)
elo_squadre=elo_squadre.set_index(["squadra","giocatore"])




## Come si svolgono gli incontri

Ogni capitano presenta la formazione (due titolari e una riserva). I due titolari vengono accoppiati agli avversari sulla base del punteggio Elo (i due più alti tra loro e i due più bassi tra loro).

Inseriscono i due racconti. Cose da controllare:
* link al racconto;
* tag;
* lunghezza del racconto;
* link al commento;
* commento valido.

Per la votazione ogni squadra presenta l'elenco dei voti da dare a quelle degli altri gironi. Per ogni partita danno quattro punteggi. Oltre a queste ci sono le penalità dell'arbitro.


In [3]:
quadro=calendario.loc[1].copy()

riposano=quadro[quadro["squadra 2"]=="(riposo)"][["squadra 1"]]
riposano



,,squadra 1
girone,partita,
A,2,Gli ultimi


In [4]:
quadro=quadro[quadro["squadra 2"]!="(riposo)"]
quadro["vs"]="contro"

quadro=quadro[["squadra 1","vs","squadra 2"]]

quadro=quadro.sort_index()
quadro

squadra 1      vs           squadra 2
girone partita                                                           
A      1                          D Euforiche  contro        La threesome
B      1                          Green House  contro          I Bananari
       2        Gli scrittori del quartierino  contro      the 3 BrOthers
C      1                        Living Stones  contro  Retroguardia Trash
       2                          Le TreMende  contro  Armata Brancaleone

In [5]:
importlib.reload(mu)
mu.to_table_html_multi(quadro)


<table style='margin:auto;' border='1' width='90%'>
<th>girone</th>
<th>partita</th>
<th>squadra 1</th>
<th>vs</th>
<th>squadra 2</th>
</tr>
<tr>
<th rowspan='1'>A</th>
<th rowspan='2'>1</th>
<td style='text-align:center;'>D Euforiche</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>La threesome</td>
</tr>
<tr>
<th rowspan='2'>B</th>
<td style='text-align:center;'>Green House</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>I Bananari</td>
</tr>
<tr>
<th rowspan='1'>2</th>
<td style='text-align:center;'>Gli scrittori del quartierino</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>the 3 BrOthers</td>
</tr>
<tr>
<th rowspan='2'>C</th>
<th rowspan='1'>1</th>
<td style='text-align:center;'>Living Stones</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>Retroguardia Trash</td>
</tr>
<tr>
<th rowspan='1'>2</th>
<td style='text-align:center;'>Le TreMende</td>
<td style='text

## Fase 1. Dalle formazioni ai match

In [6]:
squadre=scrittopoli.copy()
squadre=squadre.reset_index()[["squadra","giocatore"]].set_index("squadra")

importlib.reload(ms)
s=ms.N2017Spreadsheet()

old_data=s.get_formazioni(1)
s.set_formazioni(1,riposano["squadra 1"].values,squadre.index.unique(),old_data)


formazioni_t=s.get_formazioni(1)
formazioni_t



,riposa,capitano,riserva,titolare 1,titolare 2
squadra,,,,,
Green House,,,,,
La threesome,,,,,
I Bananari,,simone volponi,Niko,simone volponi,massimopud
Armata Brancaleone,,,,,
Living Stones,,Macleo,Ljuset,Macleo,Vincenzo Iennaco
Retroguardia Trash,,,,,
Le TreMende,,,,,
Gli scrittori del quartierino,,,,,
the 3 BrOthers,,,,,


In [7]:
# Versione random di prova, da sostituire con lettura della formazione via spreadsheet sotto
data=[]
for sq in squadre.index.unique():
    capitano=squadre.loc[sq].sample(n=1).iloc[0]["giocatore"]
    riserva=squadre.loc[sq].sample(n=1).iloc[0]["giocatore"]
    titolari=list(squadre[squadre["giocatore"]!=riserva].loc[sq]["giocatore"])
    data.append([sq,capitano,riserva]+titolari)
formazioni=pd.DataFrame(data,columns=["squadra","capitano","riserva","titolare 1","titolare 2"])
formazioni=formazioni.set_index("squadra")
formazioni["riposa"]=""
formazioni.set_value("Gli ultimi","riposa","(riposa)")

#formazioni=s.get_formazioni(1)


,capitano,riserva,titolare 1,titolare 2,riposa
squadra,,,,,
Green House,Plata,Joyopi,Bango Skank,Plata,
La threesome,Luna,mina99,Komorebi,Luna,
I Bananari,simone volponi,simone volponi,Niko,massimopud,
Armata Brancaleone,Ella F.,AdStr,Ella F.,Skorpiòs,
Living Stones,Vincenzo Iennaco,Macleo,Vincenzo Iennaco,Ljuset,
Retroguardia Trash,Eudes,AlexComan,Eudes,Andrea28,
Le TreMende,Thea,Thea,Emy,sefora,
Gli scrittori del quartierino,Federico72,camparino,Federico72,Marty12,
the 3 BrOthers,ettore70,ettore70,AryaSophia,ITG,


In [8]:
formazioni

,capitano,riserva,titolare 1,titolare 2,riposa
squadra,,,,,
Green House,Plata,Joyopi,Bango Skank,Plata,
La threesome,Luna,mina99,Komorebi,Luna,
I Bananari,simone volponi,simone volponi,Niko,massimopud,
Armata Brancaleone,Ella F.,AdStr,Ella F.,Skorpiòs,
Living Stones,Vincenzo Iennaco,Macleo,Vincenzo Iennaco,Ljuset,
Retroguardia Trash,Eudes,AlexComan,Eudes,Andrea28,
Le TreMende,Thea,Thea,Emy,sefora,
Gli scrittori del quartierino,Federico72,camparino,Federico72,Marty12,
the 3 BrOthers,ettore70,ettore70,AryaSophia,ITG,


In [9]:
importlib.reload(melo)

# c'è una componente casuale, una volta pubblicati vanno fissati
accoppiamenti=melo.calcola_accoppiamenti(1,formazioni,calendario,elo_squadre)
accoppiamenti


squadra          capitano  \
girone partita                                                    
A      1                          D Euforiche              Kuno   
       1                         La threesome              Luna   
B      1                          Green House             Plata   
       1                           I Bananari    simone volponi   
       2        Gli scrittori del quartierino        Federico72   
       2                       the 3 BrOthers          ettore70   
C      1                        Living Stones  Vincenzo Iennaco   
       1                   Retroguardia Trash             Eudes   
       2                          Le TreMende              Thea   
       2                   Armata Brancaleone           Ella F.   

                       riserva           match 1     match 2  
girone partita                                                
A      1                  Kuno            queffe    Marcello  
       1                mina99          Komorebi        Luna  
B      1                Joyopi       Bango Skank       Plata  
       1        simone volponi        massimopud        Niko  
       2             camparino           Marty12  Federico72  
       2              ettore70               ITG  AryaSophia  
C      1                Macleo  Vincenzo Iennaco      Ljuset  
       1             AlexComan          Andrea28       Eudes  
       2                  Thea               Emy      sefora  
       2                 AdStr           Ella F.    Skorpiòs

In [10]:
# Codice html per il post di riepilogo sugli accoppiamenti

prev_girone=""
for girone,partita in accoppiamenti.index.unique():
    if prev_girone!=girone:
        print("<p>&nbsp;</p>")
        print("<p><strong>Girone %s</strong></p>" % girone)
        prev_girone=girone
    p1=accoppiamenti.loc[girone,partita].iloc[0]
    p2=accoppiamenti.loc[girone,partita].iloc[1]
    print("<p>%d. %s - %s</p>" % (partita,p1["squadra"],p2["squadra"]) )
    print("<p style='margin-left: 40px;'>match 1: %s - %s</p>" % (p1["match 1"],p2["match 1"]))
    print("<p style='margin-left: 40px;'>match 2: %s - %s</p>" % (p1["match 2"],p2["match 2"]))
    

<p>&nbsp;</p>
<p><strong>Girone A</strong></p>
<p>1. D Euforiche - La threesome</p>
<p style='margin-left: 40px;'>match 1: queffe - Komorebi</p>
<p style='margin-left: 40px;'>match 2: Marcello - Luna</p>
<p>&nbsp;</p>
<p><strong>Girone B</strong></p>
<p>1. Green House - I Bananari</p>
<p style='margin-left: 40px;'>match 1: Bango Skank - massimopud</p>
<p style='margin-left: 40px;'>match 2: Plata - Niko</p>
<p>2. Gli scrittori del quartierino - the 3 BrOthers</p>
<p style='margin-left: 40px;'>match 1: Marty12 - ITG</p>
<p style='margin-left: 40px;'>match 2: Federico72 - AryaSophia</p>
<p>&nbsp;</p>
<p><strong>Girone C</strong></p>
<p>1. Living Stones - Retroguardia Trash</p>
<p style='margin-left: 40px;'>match 1: Vincenzo Iennaco - Andrea28</p>
<p style='margin-left: 40px;'>match 2: Ljuset - Eudes</p>
<p>2. Le TreMende - Armata Brancaleone</p>
<p style='margin-left: 40px;'>match 1: Emy - Ella F.</p>
<p style='margin-left: 40px;'>match 2: sefora - Skorpiòs</p>


## Fase 2. Creazione del tabellone per voti e controlli

In [11]:
importlib.reload(ms)
s=ms.N2017Spreadsheet()

# togliere quando c'è la versione definitiva delle formazioni
# old_data=s.get_giornata(1)
old_data=None
s.set_giornata(1,accoppiamenti,gironi,old_data)


## Fase 3. Recupero dati

In [12]:
importlib.reload(ms)

s=ms.N2017Spreadsheet()
voti=s.get_giornata(1)


In [13]:
### Simulazione di riempimento del tabellone - da togliere quando ci sono i voti definitivi
importlib.reload(mu)

voti["base","titolo"]=voti["base","titolo"].apply(mu.random_title)
voti["base","ingresso riserva"]=voti["base","ingresso riserva"].apply(mu.random_riserva)
voti["verifiche","lunghezza"]=voti["verifiche","lunghezza"].apply(mu.random_len)
voti["verifiche","link"]=voti["verifiche","link"].apply(mu.random_verifica)
voti["verifiche","link commento"]=voti["verifiche","link commento"].apply(mu.random_verifica)
voti["verifiche","tag"]=voti["verifiche","tag"].apply(mu.random_verifica)
voti["verifiche","commento valido"]=voti["verifiche","commento valido"].apply(mu.random_verifica)

for G,altri in [ ("A",[("B",1),("B",2),("C",1),("C",2)]),
                 ("B",[("A",1),("C",1),("C",2)]),
                 ("C",[("A",1),("B",1),("B",2)]) ]:
    for sq in gironi[gironi[G]!="(riposo)"][G]:
        for gir,part in altri:
            for m in ["match 1","match 2"]:
                giocatore_1=accoppiamenti.loc[gir,part][m].iloc[0]
                giocatore_2=accoppiamenti.loc[gir,part][m].iloc[1]
                goal_1=random.randint(0,2)
                goal_2=2-goal_1
                voti.set_value(giocatore_1,(G,sq),goal_1)
                voti.set_value(giocatore_2,(G,sq),goal_2)

voti["verifiche","penalità"]=voti["verifiche","lunghezza"].apply(mu.calc_penalita_lungh)
for k in ["tag","link","link commento","commento valido"]:
    voti["verifiche","penalità"]+=voti["verifiche",k].apply(mu.calc_penalita)


In [14]:
voti["base","goal giocatore"]=voti[["A","B","C"]].sum(axis=1,level=1).sum(axis=1)+voti["verifiche","penalità"]
voti

base                       verifiche           \
                 ingresso riserva                titolo lunghezza tag link   
titolare                                                                     
queffe                12-08 23:23          WIRkgxJcWKbF      2788   x    x   
Marcello                           TO z6Wu0GH CvOxnXDHK      3249   x        
Komorebi              12-08 23:23     S9qrfuPcAq0hTH3kg       634   x    x   
Luna                  12-08 23:23           kTII 4qoEOo      7242   x    x   
Bango Skank           12-08 23:23    iMo1MPt Q 1K OO aI      6863   x    x   
Plata                 12-08 23:23          6 1 bvjcv6QP      2939   x    x   
massimopud                                    5vHD r1if      4418   x        
Niko                  12-08 23:23             5MG33CH N      5111        x   
Marty12                                OT mk6PPtM0zh8hE      4271   x    x   
Federico72                                 UH ii74dJ aZ      5958   x    x   
ITG                   12-08 23:23    J4i C qY Mzckeu VJ      7423   x    x   
AryaSophia            12-08 23:23           uMN8p Cs9MM      4734   x    x   
Vincenzo Iennaco      12-08 23:23        JlXs43 2I7lTXT      6319   x    x   
Ljuset                                      z lReNPetL6      7526   x    x   
Andrea28                              U c2yR PDb W7 d6c      2067   x    x   
Eudes                 12-08 23:23            ZzNuThuh F      4582   x    x   
Emy                                       lR4 g4raaJZit      5685   x    x   
sefora                12-08 23:23     XFanTSLFqvEv gX q      7084   x    x   
Ella F.                                     Rtf H iJlIz      1708   x    x   
Skorpiòs                                      I2hPJRO a      3466   x    x   

                                                                  A  \
                 link commento commento valido penalità D Euforiche   
titolare                                                              
queffe                       x               x        0           0   
Marcello                     x                       -2           0   
Komorebi                     x               x        0           0   
Luna                                                 -2           0   
Bango Skank                  x               x        0           1   
Plata                        x               x        0           1   
massimopud                   x               x       -1           1   
Niko                         x               x       -1           1   
Marty12                      x               x        0           2   
Federico72                   x               x        0           1   
ITG                          x               x        0           0   
AryaSophia                   x               x        0           1   
Vincenzo Iennaco             x                       -1           1   
Ljuset                       x               x        0           1   
Andrea28                                     x       -1           1   
Eudes                        x               x        0           1   
Emy                          x               x        0           0   
sefora                       x               x        0           1   
Ella F.                      x               x        0           2   
Skorpiòs                     x               x        0           1   

                                                   B             \
                 La threesome Gli ultimi Green House I Bananari   
titolare                                                          
queffe                      0          0           1          1   
Marcello                    0          0           0          2   
Komorebi                    0          0           1          1   
Luna                        0          0           2          0   
Bango Skank                 0          2           0          0   
Plata                       2          2           0          0 

## Calcolo dei risultati

### Risultati della prima giornata

In [15]:
risultati=accoppiamenti.copy()

risultati=risultati.reset_index().set_index("squadra")
risultati["match 1 goal"]=0
risultati["match 1 riserva"]=False
risultati["match 2 goal"]=0
risultati["match 2 riserva"]=False
risultati["goal"]=0
risultati["penalità"]=0


for sq in risultati.index:
    m1=risultati.loc[sq]["match 1"]
    m2=risultati.loc[sq]["match 2"]
    goal_1=voti.loc[m1]["base","goal giocatore"]
    goal_2=voti.loc[m2]["base","goal giocatore"]
    pen_1=voti.loc[m1]["verifiche","penalità"]
    pen_2=voti.loc[m2]["verifiche","penalità"]
    risultati.set_value(sq,"match 1 goal",goal_1)
    risultati.set_value(sq,"match 2 goal",goal_2)
    risultati.set_value(sq,"goal",goal_1+goal_2)
    risultati.set_value(sq,"penalità",pen_1+pen_2)
    ris_1=bool(voti.loc[m1]["base","ingresso riserva"])
    risultati.set_value(sq,"match 1 riserva",ris_1)
    if ris_1: continue
    risultati.set_value(sq,"match 2 riserva",bool(voti.loc[m2]["base","ingresso riserva"]))
    
risultati=risultati.reset_index().set_index(["girone","partita"])
sc_dati=[]
for girone,partita in risultati.index.unique():
    squadra_1=risultati.loc[girone,partita]["squadra"].iloc[0]
    squadra_2=risultati.loc[girone,partita]["squadra"].iloc[1]
    goal_1=risultati.loc[girone,partita]["goal"].iloc[0]
    goal_2=risultati.loc[girone,partita]["goal"].iloc[1]
    pen_1=risultati.loc[girone,partita]["penalità"].iloc[0]
    pen_2=risultati.loc[girone,partita]["penalità"].iloc[1]
    m1_goal_1=risultati.loc[girone,partita]["match 1 goal"].iloc[0]
    m1_goal_2=risultati.loc[girone,partita]["match 1 goal"].iloc[1]
    m2_goal_1=risultati.loc[girone,partita]["match 2 goal"].iloc[0]
    m2_goal_2=risultati.loc[girone,partita]["match 2 goal"].iloc[1]
    sc_dati.append([girone,partita,squadra_1,squadra_2,goal_1,goal_2,
                    m1_goal_1,m1_goal_2,m2_goal_1,m2_goal_2,pen_1,pen_2])

risultati=risultati.reset_index().set_index(["girone","partita","squadra"])

schedina=pd.DataFrame(sc_dati,columns=["girone","partita","squadra 1","squadra 2",
                                       "goal squadra 1","goal squadra 2",
                                       "match 1 squadra 1","match 1 squadra 2",
                                       "match 2 squadra 1","match 2 squadra 2",
                                       "penalità squadra 1","penalità squadra 2"])

out_schedina=schedina.copy()
out_schedina["titolo"]=out_schedina["squadra 1"]+" - "+out_schedina["squadra 2"]
out_schedina["risultato"]=( out_schedina["goal squadra 1"].apply(str)
                            +" - "+out_schedina["goal squadra 2"].apply(str) )
out_schedina["match 1"]=( out_schedina["match 1 squadra 1"].apply(str)
                          +" - "+out_schedina["match 1 squadra 2"].apply(str) )
out_schedina["match 2"]=( out_schedina["match 2 squadra 1"].apply(str)
                          +" - "+out_schedina["match 2 squadra 2"].apply(str) )
out_schedina=out_schedina.set_index(["girone","partita"])

out_schedina=out_schedina[["titolo","match 1","match 2","risultato"]]


risultati

capitano  \
girone partita squadra                                           
A      1       D Euforiche                                Kuno   
               La threesome                               Luna   
B      1       Green House                               Plata   
               I Bananari                       simone volponi   
       2       Gli scrittori del quartierino        Federico72   
               the 3 BrOthers                         ettore70   
C      1       Living Stones                  Vincenzo Iennaco   
               Retroguardia Trash                        Eudes   
       2       Le TreMende                                Thea   
               Armata Brancaleone                      Ella F.   

                                                     riserva  \
girone partita squadra                                         
A      1       D Euforiche                              Kuno   
               La threesome                           mina99   
B      1       Green House                            Joyopi   
               I Bananari                     simone volponi   
       2       Gli scrittori del quartierino       camparino   
               the 3 BrOthers                       ettore70   
C      1       Living Stones                          Macleo   
               Retroguardia Trash                  AlexComan   
       2       Le TreMende                              Thea   
               Armata Brancaleone                      AdStr   

                                                       match 1     match 2  \
girone partita squadra                                                       
A      1       D Euforiche                              queffe    Marcello   
               La threesome                           Komorebi        Luna   
B      1       Green House                         Bango Skank       Plata   
               I Bananari                           massimopud        Niko   
       2       Gli scrittori del quartierino           Marty12  Federico72   
               the 3 BrOthers                              ITG  AryaSophia   
C      1       Living Stones                  Vincenzo Iennaco      Ljuset   
               Retroguardia Trash                     Andrea28       Eudes   
       2       Le TreMende                                 Emy      sefora   
               Armata Brancaleone                      Ella F.    Skorpiòs   

                                              match 1 goal  match 1 riserva  \
girone partita squadra                                                        
A      1       D Euforiche                               4             True   
               La threesome                             12             True   
B      1       Green House                               8             True   
               I Bananari                                5            False   
       2       Gli scrittori del quartierino            12            False   
               the 3 BrOthers                            2             True   
C      1       Living Stones                            10             True   
               Retroguardia Trash                        2            False   
       2       Le TreMende                               7            False   
               Armata Brancaleone                        7            False   

                                              match 2 goal  match 2 riserva  \
girone partita squadra                                                        
A      1       D Euforiche                               9            False   
               La threesome                              3            False   
B      1       Green House                              10            False   
               I Bananari                                3             True   
       2       Gli scrittori del quartierino             5            False   
               the 3 BrOthers                        

In [16]:
schedina

,girone,partita,squadra 1,squadra 2,goal squadra 1,goal squadra 2,match 1 squadra 1,match 1 squadra 2,match 2 squadra 1,match 2 squadra 2,penalità squadra 1,penalità squadra 2
0,A,1,D Euforiche,La threesome,13,15,4,12,9,3,-2,-2
1,B,1,Green House,I Bananari,18,8,8,5,10,3,0,-2
2,B,2,Gli scrittori del quartierino,the 3 BrOthers,17,11,12,2,5,9,0,0
3,C,1,Living Stones,Retroguardia Trash,18,8,10,2,8,6,-1,-1
4,C,2,Le TreMende,Armata Brancaleone,13,15,7,7,6,8,0,0


In [17]:
out_schedina

titolo match 1  \
girone partita                                                           
A      1                            D Euforiche - La threesome  4 - 12   
B      1                              Green House - I Bananari   8 - 5   
       2        Gli scrittori del quartierino - the 3 BrOthers  12 - 2   
C      1                    Living Stones - Retroguardia Trash  10 - 2   
       2                      Le TreMende - Armata Brancaleone   7 - 7   

               match 2 risultato  
girone partita                    
A      1         9 - 3   13 - 15  
B      1        10 - 3    18 - 8  
       2         5 - 9   17 - 11  
C      1         8 - 6    18 - 8  
       2         6 - 8   13 - 15

In [18]:
# output per il post dei risultati

mu.to_table_html_multi(out_schedina)


<table style='margin:auto;' border='1' width='90%'>
<th>girone</th>
<th>partita</th>
<th>titolo</th>
<th>match 1</th>
<th>match 2</th>
<th>risultato</th>
</tr>
<tr>
<th rowspan='1'>A</th>
<th rowspan='2'>1</th>
<td style='text-align:center;'>D Euforiche - La threesome</td>
<td style='text-align:center;'>4 - 12</td>
<td style='text-align:center;'>9 - 3</td>
<td style='text-align:center;'>13 - 15</td>
</tr>
<tr>
<th rowspan='2'>B</th>
<td style='text-align:center;'>Green House - I Bananari</td>
<td style='text-align:center;'>8 - 5</td>
<td style='text-align:center;'>10 - 3</td>
<td style='text-align:center;'>18 - 8</td>
</tr>
<tr>
<th rowspan='1'>2</th>
<td style='text-align:center;'>Gli scrittori del quartierino - the 3 BrOthers</td>
<td style='text-align:center;'>12 - 2</td>
<td style='text-align:center;'>5 - 9</td>
<td style='text-align:center;'>17 - 11</td>
</tr>
<tr>
<th rowspan='2'>C</th>
<th rowspan='1'>1</th>
<td style='text-align:center;'>Living Stones - Retroguardia Trash</td>


### Classifica


Ordine dei criteri:
* classifiche avulse (scontri diretti);
* differenza reti;
* numero di reti segnate;
* numero di vittorie;
* differenza reti nell'ultimo incontro;
* numero di reti segnate nell'ultimo incontro;
* differenza reti al netto di goal assegnati o tolti dall'arbitro;
* numero di reti segnate al netto di goal assegnati o tolti dall'arbitro.

Nella prima giornata non ci sono classifiche avulse e l'unico incontro è anche l'ultimo, quindi si considerano solo:
* differenza reti;
* numero di reti segnate;
* numero di vittorie;
* differenza reti al netto di goal assegnati o tolti dall'arbitro;
* numero di reti segnate al netto di goal assegnati o tolti dall'arbitro.


In [19]:
q=schedina.copy()

#q.merge(q,left_on="squadra 1",right_on="squadra 2",suffixes=(" sq1"," sq2"),how="outer")

a=q[["girone","squadra 1","goal squadra 1","goal squadra 2",
     "penalità squadra 1","penalità squadra 2"]].copy()
b=q[["girone","squadra 2","goal squadra 2","goal squadra 1",
     "penalità squadra 2","penalità squadra 1"]].copy()

a.columns=["girone","squadra","goal fatti","goal subiti","penalità fatte","penalità subite"]
b.columns=["girone","squadra","goal fatti","goal subiti","penalità fatte","penalità subite"]

classifica=a.append(b).set_index(["girone","squadra"]).sort_index()

classifica["goal netti fatti"]=classifica["goal fatti"]-classifica["penalità fatte"]
classifica["goal netti subiti"]=classifica["goal subiti"]-classifica["penalità subite"]

classifica["differenza reti"]=classifica["goal fatti"]-classifica["goal subiti"]
classifica["differenza reti netta"]=classifica["goal netti fatti"]-classifica["goal netti subiti"]

def calc_vittorie(row):
    row["vittorie"]=0
    row["sconfitte"]=0
    row["pareggi"]=0
    if row["goal fatti"]>row["goal subiti"]:
        row["vittorie"]=1
    elif row["goal fatti"]<row["goal subiti"]:
        row["sconfitte"]=1
    else:
        row["pareggi"]=1
    row["punti"]=3*row["vittorie"]+row["pareggi"]
    return row

classifica=classifica.apply(calc_vittorie,axis=1)


df=pd.DataFrame([["A","Gli ultimi",0]],columns=["girone","squadra","goal fatti"])
df=df.set_index(["girone","squadra"])
classifica=classifica.append(df).fillna(0).sort_index()
classifica=classifica[["punti","vittorie","pareggi","sconfitte","differenza reti",
                   "goal fatti","goal subiti",
                   "differenza reti netta",
                   "goal netti fatti","goal netti subiti","penalità fatte","penalità subite"]]
classifica=classifica.astype(int)
classifica=classifica.sort_values(by=["punti","differenza reti","goal fatti","vittorie",
                                  "differenza reti netta","goal netti fatti"],
                              ascending=False).sort_index(level="girone",
                                                          kind="mergesort",
                                                          sort_remaining=False)
classifica

punti  vittorie  pareggi  sconfitte  \
girone squadra                                                              
A      La threesome                       3         1        0          0   
       Gli ultimi                         0         0        0          0   
       D Euforiche                        0         0        0          1   
B      Green House                        3         1        0          0   
       Gli scrittori del quartierino      3         1        0          0   
       the 3 BrOthers                     0         0        0          1   
       I Bananari                         0         0        0          1   
C      Living Stones                      3         1        0          0   
       Armata Brancaleone                 3         1        0          0   
       Le TreMende                        0         0        0          1   
       Retroguardia Trash                 0         0        0          1   

                                      differenza reti  goal fatti  \
girone squadra                                                      
A      La threesome                                 2          15   
       Gli ultimi                                   0           0   
       D Euforiche                                 -2          13   
B      Green House                                 10          18   
       Gli scrittori del quartierino                6          17   
       the 3 BrOthers                              -6          11   
       I Bananari                                 -10           8   
C      Living Stones                               10          18   
       Armata Brancaleone                           2          15   
       Le TreMende                                 -2          13   
       Retroguardia Trash                         -10           8   

                                      goal subiti  differenza reti netta  \
girone squadra                                                             
A      La threesome                            13                      2   
       Gli ultimi                               0                      0   
       D Euforiche                             15                     -2   
B      Green House                              8                      8   
       Gli scrittori del quartierino           11                      6   
       the 3 BrOthers                          17                     -6   
       I Bananari                              18                     -8   
C      Living Stones                            8                     10   
       Armata Brancaleone                      13                      2   
       Le TreMende                             15                     -2   
       Retroguardia Trash                      18                    -10   

                                      goal netti fatti  goal netti subiti  \
girone squadra                                                              
A      La threesome                                 17                 15   
       Gli ultimi                                    0                  0   
       D Euforiche                                  15                 17   
B      Green House                                  18                 10   
       Gli scrittori del quartierino                17                 11   
       the 3 BrOthers                               11                 17   
       I Bananari                                   10                 18   
C      Living Stones                                19                  9   
       Armata Brancaleone                           15                 13   
       Le TreMende                                  13                 15   
       Retroguardia Trash                            9                 19   

                                      penalità fatte  penalità subite  
girone squadra                                                         
A  

In [20]:
## Output per il post della classifica

mu.to_table_html_multi(classifica)

<table style='margin:auto;' border='1' width='90%'>
<th>girone</th>
<th>squadra</th>
<th>punti</th>
<th>vittorie</th>
<th>pareggi</th>
<th>sconfitte</th>
<th>differenza reti</th>
<th>goal fatti</th>
<th>goal subiti</th>
<th>differenza reti netta</th>
<th>goal netti fatti</th>
<th>goal netti subiti</th>
<th>penalità fatte</th>
<th>penalità subite</th>
</tr>
<tr>
<th rowspan='3'>A</th>
<th rowspan='1'>La threesome</th>
<td style='text-align:center;'>3</td>
<td style='text-align:center;'>1</td>
<td style='text-align:center;'>0</td>
<td style='text-align:center;'>0</td>
<td style='text-align:center;'>2</td>
<td style='text-align:center;'>15</td>
<td style='text-align:center;'>13</td>
<td style='text-align:center;'>2</td>
<td style='text-align:center;'>17</td>
<td style='text-align:center;'>15</td>
<td style='text-align:center;'>-2</td>
<td style='text-align:center;'>-2</td>
</tr>
<tr>
<th rowspan='1'>Gli ultimi</th>
<td style='text-align:center;'>0</td>
<td style='text-align:center;'>0</td

### Nuovi punteggi Elo

In [21]:
q=risultati.copy()

def calc_giocatore(row,match):
    riserva=row["riserva"]
    giocatore=row["match %d" % match]
    sostituzione=row["match %d riserva" % match]
    if sostituzione: return riserva
    return giocatore

def calc_giocatore_1(row): return calc_giocatore(row,1)
def calc_giocatore_2(row): return calc_giocatore(row,2)

q["giocatore 1"]=q.apply(calc_giocatore_1,axis=1)
q["giocatore 2"]=q.apply(calc_giocatore_2,axis=1)

a=q[["giocatore 1","match 1 goal"]].copy()
a.columns=["giocatore","goal"]
a["match"]="1"

b=q[["giocatore 2","match 2 goal"]].copy()
b.columns=["giocatore","goal"]
b["match"]="2"

q=a.append(b).reset_index().set_index(["girone","partita","match"])
q=q.merge(q,how="outer",left_index=True,right_index=True,suffixes=[" 1"," 2"])
q=q[q["giocatore 1"]!=q["giocatore 2"]]

elo=q[["squadra 1","giocatore 1","giocatore 2","goal 1","goal 2"]]
elo.columns=["squadra","giocatore","avversario","goal fatti","goal subiti"]
elo=elo.reset_index().set_index(["girone","squadra","giocatore"]).sort_index()
elo=elo[["avversario","goal fatti","goal subiti"]].copy()
elo["elo precedente"]=1500
elo["elo avversario"]=1500

def calcola_punteggi(row):
    exp_a,exp_b=melo.punteggi_attesi(row["elo precedente"],row["elo avversario"])
    score_a,score_b=melo.punteggi_reali(row["goal fatti"],row["goal subiti"])
    row["punteggio atteso"]=exp_a
    row["punteggio reale"]=score_a
    return row

elo=elo.apply(calcola_punteggi,axis=1)

elo["scostamento"]=elo["punteggio reale"]-elo["punteggio atteso"]
mean=elo["scostamento"].mean()
sigma=elo["scostamento"].std()

for alpha in np.linspace(0,6,601):
    if mlab.normpdf(alpha*sigma,mean,sigma)<=0.0001:
        break

#x = np.linspace(mean - alpha*sigma, mean + alpha*sigma, 100)
#plt.plot(x,mlab.normpdf(x, mean, sigma))
#plt.show()

max_scostamento=alpha*sigma
max_variazione=50
K=max_variazione/max_scostamento

def norm_elo(x):
    e=round(max(1450,x))
    return int(e)
    
elo["elo nuovo"]=elo["elo precedente"]+K*elo["scostamento"]
elo["elo nuovo"]=elo["elo nuovo"].apply(norm_elo)
elo=elo.reset_index().set_index(["giocatore"]).sort_values("elo nuovo",ascending=False)
elo

,girone,squadra,avversario,goal fatti,goal subiti,elo precedente,elo avversario,punteggio atteso,punteggio reale,scostamento,elo nuovo
giocatore,,,,,,,,,,,
Marty12,B,Gli scrittori del quartierino,ettore70,12,2,1500,1500,0.5,0.857143,0.357143,1518
Macleo,C,Living Stones,Andrea28,10,2,1500,1500,0.5,0.833333,0.333333,1517
Plata,B,Green House,simone volponi,10,3,1500,1500,0.5,0.769231,0.269231,1514
mina99,A,La threesome,Kuno,12,4,1500,1500,0.5,0.750000,0.250000,1513
Marcello,A,D Euforiche,Luna,9,3,1500,1500,0.5,0.750000,0.250000,1513
AryaSophia,B,the 3 BrOthers,Federico72,9,5,1500,1500,0.5,0.642857,0.142857,1507
Joyopi,B,Green House,massimopud,8,5,1500,1500,0.5,0.615385,0.115385,1506
Ljuset,C,Living Stones,AlexComan,8,6,1500,1500,0.5,0.571429,0.071429,1504
Skorpiòs,C,Armata Brancaleone,Thea,8,6,1500,1500,0.5,0.571429,0.071429,1504


In [22]:
a=gironi[gironi["A"]!="(riposo)"][["A"]].copy()
a.columns=["squadra"]
a["girone"]="A"
a

b=gironi[gironi["B"]!="(riposo)"][["B"]].copy()
b.columns=["squadra"]
b["girone"]="B"
b

c=gironi[gironi["C"]!="(riposo)"][["C"]].copy()
c.columns=["squadra"]
c["girone"]="C"
c

q=a.append(b).append(c).set_index("squadra")
q=q.merge(squadre,left_index=True,right_index=True,how="outer")
q=q.reset_index().set_index("giocatore")
q

elo_g=elo.merge(q,how="outer",
                left_index=True,right_index=True)
elo_g=elo_g[["squadra_y","elo nuovo"]].fillna(1500).sort_values("elo nuovo",
                                                                           ascending=False)
elo_g["elo nuovo"]=pd.to_numeric(elo_g["elo nuovo"],downcast="signed")
elo_g.columns=["squadra","elo"]
elo_g

,squadra,elo
giocatore,,
Marty12,Gli scrittori del quartierino,1518
Macleo,Living Stones,1517
Plata,Green House,1514
Marcello,D Euforiche,1513
mina99,La threesome,1513
AryaSophia,the 3 BrOthers,1507
Joyopi,Green House,1506
Skorpiòs,Armata Brancaleone,1504
Ljuset,Living Stones,1504


In [23]:
mu.to_table_html(elo_g,with_index=True)

<table style='margin:auto;' border='1' width='90%'>
<tr><th>&nbsp;</th>
<th>squadra</th>
<th>elo</th>
</tr>
<tr>
<th>Marty12</th>
<td style='text-align:center;'>Gli scrittori del quartierino</td>
<td style='text-align:center;'>1518</td>
</tr>
<tr>
<th>Macleo</th>
<td style='text-align:center;'>Living Stones</td>
<td style='text-align:center;'>1517</td>
</tr>
<tr>
<th>Plata</th>
<td style='text-align:center;'>Green House</td>
<td style='text-align:center;'>1514</td>
</tr>
<tr>
<th>Marcello</th>
<td style='text-align:center;'>D Euforiche</td>
<td style='text-align:center;'>1513</td>
</tr>
<tr>
<th>mina99</th>
<td style='text-align:center;'>La threesome</td>
<td style='text-align:center;'>1513</td>
</tr>
<tr>
<th>AryaSophia</th>
<td style='text-align:center;'>the 3 BrOthers</td>
<td style='text-align:center;'>1507</td>
</tr>
<tr>
<th>Joyopi</th>
<td style='text-align:center;'>Green House</td>
<td style='text-align:center;'>1506</td>
</tr>
<tr>
<th>Skorpiòs</th>
<td style='text-align:cente

## Salvataggio dei risultati

In [24]:
## CSV
label="giornata_1"

# formazioni
formazioni_csv="data/formazioni_"+label+".csv"
formazioni.to_csv(formazioni_csv,index_label="squadra")

# accoppiamenti
accoppiamenti_csv="data/accoppiamenti_"+label+".csv"
accoppiamenti.to_csv(accoppiamenti_csv)

# voti
voti_csv="data/voti_"+label+".csv"
voti.to_csv(voti_csv)

# risultati
risultati_csv="data/risultati_"+label+".csv"
risultati.to_csv(risultati_csv)

# schedina
schedina_csv="data/schedina_"+label+".csv"
schedina.to_csv(schedina_csv,index=False)

# classifica
classifica_csv="data/classifica_"+label+".csv"
classifica.to_csv(classifica_csv)

# elo
elo_csv="data/elo_"+label+".csv"
elo.to_csv(elo_csv)

# elo_g
elo_g_csv="data/elo_g_"+label+".csv"
elo_g.to_csv(elo_g_csv)

